In [256]:
# Modules
import pandas as pd
import requests
from datetime import datetime, timezone # for time column and EST timezone conversion
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [257]:
# API URL and params in case request gets denied
api_url = "https://developer.nrel.gov/api/pvwatts/v8.json" # endpoint with the argivoltaics data
params = {
    'api_key': 'LMpKxIwqBHBN9TJ9x4qtzhR2cD6mQUpZXBAURgbb',
    'system_capacity' : 0.05,
    'module_type' : 0,
    'losses' : 0,
    'array_type' : 0,
    'tilt' : 0,
    'azimuth' : 0,
    'lat' : 38.972360, # Maryland latitude
    'lon' : -76.937732, # Maryland longitude
    'timeframe' : 'hourly' # dataset is hourly, not monthly or annually
}

In [258]:
response = requests.get(api_url, params = params)
json = response.json()
outputs = json['outputs']
current_time = datetime.now(timezone.utc)
length = len(outputs['ac'])
# useful to know : 720 hours is a month, dataset shifts every hour
times = pd.date_range(end=current_time, periods=length ,freq='h')
hourly_data = {
    'Time EST' : times,
    'Hourly alternating current system output(Wac)' : outputs['ac'],
    'Hourly plane of array irradiance(W/m2)' : outputs['poa'],
    'Hourly beam normal irradiance(W/m2)' : outputs['dn'],
    'Hourly direct current array output(Wdc)' : outputs['dc'],
    'Hourly diffuse irradiance(W/m2)' : outputs['df'],
    'Hourly ambient temperature(C)' : outputs['tamb'],
    'Hourly module temperature(C)' : outputs['tcell'],
    'Hourly windspeed(m/s)' : outputs['wspd'],
    'Hourly albedo values' : outputs['alb']
} # created dictionary with hourly data from the dataset
df = pd.DataFrame(hourly_data) # created dataframe with the dictionary
df['Time EST'] = df['Time EST'].dt.tz_convert('US/Eastern') # want the time to reflect the current time (EST)
df.set_index('Time EST', inplace=True) # make the time the first column
df.to_csv('agrivoltaics.csv', index=True) # CSV file created